This notebook was created by Donna Faith Go.

In [11]:
# standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

# data gathering
import yfinance as yf
import time
import pandas_datareader.data as web
from datetime import datetime, timedelta

## Closing Prices

## Stock Tickers

In [3]:
filepath = 'SSE companies list.pkl'
with open(filepath, 'rb') as f:
    companies_df = pickle.load(f)
companies_df.head()

,Company Name,Company Code
0,"SHANGHAI PUDONG DEVELOPMENT BANK CO., LTD.",600000
1,"Guangzhou Baiyun International Airport Co.,ltd.",600004
2,"Dongfeng Automobile Co.,LTD",600006
3,China World Trade Center Company Ltd.,600007
4,Beijing Capital Eco-Environment Protection Gro...,600008


In [8]:
tickers_list = companies_df['Company Code'].to_list()
tickers_list = [ticker + ".SS" for ticker in tickers_list]
print(tickers_list[:5])

['600000.SS', '600004.SS', '600006.SS', '600007.SS', '600008.SS']


## yfinance

In [12]:
# getting closing prices for the 30 stocks with batching
start_date = '2000-01-01'
end_date = '2026-01-01'

def download_stocks_in_batches(tickers, batch_size=5, delay=1, verbose=False):
    """
    Download stock data in batches to avoid rate limiting
    """
    all_data = {}
    
    for i in range(0, len(tickers), batch_size):
        batch = tickers[i:i + batch_size]
        if verbose:
            print(f"Downloading batch {i//batch_size + 1}: {batch}")
        
        try:
            # Download the batch
            batch_data = yf.download(
                batch,
                start=start_date,
                end=end_date,
                progress=False
            )
            
            # Extract closing prices for this batch
            if not batch_data.empty and 'Close' in batch_data.columns:
                closes = batch_data['Close']
                if isinstance(closes, pd.Series):
                    all_data[batch[0]] = closes
                else:
                    for ticker in closes.columns:
                        all_data[ticker] = closes[ticker]
                if verbose:
                    print(f"Successfully downloaded {len(batch)} stocks")
            else:
                print(f"No data returned for batch: {batch}")
            
        except Exception as e:
            print(f"Error downloading batch {batch}: {e}")
        
        # Add delay to avoid rate limiting
        if i + batch_size < len(tickers):
            if verbose:
                print(f"Waiting {delay} seconds before next batch...")
            time.sleep(delay)
    
    if all_data:
        return pd.DataFrame(all_data)
    else:
        return pd.DataFrame()

In [14]:
# Download the closing prices
closing_df = download_stocks_in_batches(
    tickers_list, 
    batch_size=5, 
    delay=5
)

# removing unnecessary columns and rows
closing_df.dropna(how='all', axis=1, inplace=True)
closing_df.dropna(how='all', axis=0, inplace=True)

# # save to pkl
# if not closing_df.empty:
#     closing_df.to_pickle('SSE companies closing prices.pkl')

# cut into parts
closing_df1 = closing_df.iloc[:, :int(np.floor(len(closing_df) / 2))]
closing_df2 = closing_df.iloc[:, int(np.floor(len(closing_df) / 2)):]

# save as pkl files
if not closing_df1.empty and closing_df2.empty:
    closing_df1.to_pickle('data/01 SSE companies closing prices.pkl')
    closing_df2.to_pickle('data/02 SSE companies closing prices.pkl')

C:\Users\Donna\AppData\Local\Temp\ipykernel_3676\2992744886.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  batch_data = yf.download(


Successfully downloaded 5 stocks
Waiting 5 seconds before next batch...


C:\Users\Donna\AppData\Local\Temp\ipykernel_3676\2992744886.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  batch_data = yf.download(


Successfully downloaded 5 stocks
Waiting 5 seconds before next batch...


C:\Users\Donna\AppData\Local\Temp\ipykernel_3676\2992744886.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  batch_data = yf.download(


...

C:\Users\Donna\AppData\Local\Temp\ipykernel_3676\2992744886.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  batch_data = yf.download(


Successfully downloaded 5 stocks
Waiting 5 seconds before next batch...


C:\Users\Donna\AppData\Local\Temp\ipykernel_3676\2992744886.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  batch_data = yf.download(


Successfully downloaded 5 stocks
Waiting 5 seconds before next batch...


KeyboardInterrupt: 

Note: The code above takes around 30-45 minutes to completely run. 